In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate, Input, Flatten, LeakyReLU,BatchNormalization
from keras.models import Model
from keras import optimizers
from keras import regularizers
import json
import re

Using TensorFlow backend.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ivan/pro

In [2]:
def jsons_name_type(jsons_name):
    if not jsons_name:
        return -1
    if "Цифра дня" in jsons_name:
        return 1
    if "AliExpress" in jsons_name:
        return 2 
    if "Тинькофф" in jsons_name or "Tinkoff" in jsons_name:
        return 3
    if "ЧМ" in jsons_name:
        return 4
    if "фильм"  in jsons_name or "смотрет"  in jsons_name:
        return 5
    if "кредит"  in jsons_name:
        return 6
    if "инвестиц"  in jsons_name:
        return 7
    if "₽"  in jsons_name or "$"  in jsons_name:
        return 8
    if "тпуск"  in jsons_name:
        return 9
    return 0

In [3]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_train['first_session_dttm']=pd.to_datetime(customer_train['first_session_dttm'],infer_datetime_format=True)
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
customer_test['first_session_dttm']=pd.to_datetime(customer_test['first_session_dttm'],infer_datetime_format=True)
stories_description = pd.read_csv(f'{data_path}stories_description.csv')

stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [4]:

subsets = stories_reaction_train.groupby(['customer_id', 'event_dttm']).count().reset_index()[['customer_id', 'event_dttm', 'story_id']]
subsets.columns = ['customer_id', 'event_dttm','subsets']
stories_reaction_train=stories_reaction_train.merge(subsets, on=['customer_id', 'event_dttm'], how='left')

subsets = stories_reaction_test.groupby(['customer_id', 'event_dttm']).count().reset_index()[['customer_id', 'event_dttm', 'story_id']]
subsets.columns = ['customer_id', 'event_dttm','subsets']
stories_reaction_test=stories_reaction_test.merge(subsets, on=['customer_id', 'event_dttm'], how='left')

In [5]:
stories_reaction_train['reaction_hour']=stories_reaction_train.event_dttm.dt.hour
stories_reaction_test['reaction_hour']=stories_reaction_test.event_dttm.dt.hour
stories_reaction_train['reaction_dayofweek']=stories_reaction_train.event_dttm.dt.dayofweek
stories_reaction_test['reaction_dayofweek']=stories_reaction_test.event_dttm.dt.dayofweek

stories_description['jsons'] = stories_description.story_json.map(lambda x:json.loads(x.replace("\\\\", "\\")))
stories_description['jsons_name'] =stories_description.jsons.map(lambda x:x['properties'].get('name'))
stories_description['texts'] =stories_description.jsons.map(lambda x:re.findall(""""text": \"([^\"]*)\",""", json.dumps(x, ensure_ascii=False)))
stories_description['jsons_name_type'] = stories_description.jsons_name.map(lambda x:jsons_name_type(x))

In [6]:
customer_train['first_session_dttm_year']=(customer_train.first_session_dttm.dt.year-2000)/20


In [7]:
train_and_val = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
train_and_val = train_and_val.merge(stories_description, on='story_id', how='left')

test = stories_reaction_test.merge(customer_test, on='customer_id', how='left')
test = test.merge(stories_description, on='story_id', how='left')

train_and_val=train_and_val.fillna(0)
# train_and_val.fillnull(0)


In [8]:
train_dates = stories_reaction_train['event_dttm'].astype(np.int64)
test_dates = stories_reaction_test['event_dttm'].astype(np.int64)
period = train_dates.max() - (test_dates.max() - test_dates.min())

# np.random.seed(19)
# msk = np.random.rand(len(train_and_val)) < 0.8
# train = train_and_val[msk]
# val = train_and_val[~msk]

val = train_and_val[train_and_val.event_dttm.astype(np.int64) > period]
train = train_and_val[train_and_val.event_dttm.astype(np.int64) <= period]

In [9]:
# customer_shows = train.groupby('customer_id')['event'].count().to_frame().reset_index() 
# customer_shows.columns = ['customer_id', 'customer_shows']
# customer_shows=customer_shows.fillna(0)
# train=train.merge(customer_shows, on='customer_id')
# val=val.merge(customer_shows, on='customer_id', how='left')
# train['customer_shows']=train.customer_shows.fillna(-999)
# val['customer_shows']=val.customer_shows.fillna(-999)

In [10]:
len(train), len(val) 

(320248, 152893)

In [11]:

train['gender_cd']=train.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
val['gender_cd']=val.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
val['children_cnt']=val.children_cnt/10
train['children_cnt']=train.children_cnt/10


/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [12]:
def to_OHE(field):
    count = len(set(train_and_val[field].values))
    d = dict(zip(list(set(train_and_val[field].values)), range(1, count+1)))
    train[field]=train[field].map(lambda x:  d[x])
    val[field]=val[field].map(lambda x:  d[x])
    return count
customers_count = to_OHE('customer_id')
# marital_status_count = to_OHE('marital_status_cd')
stories_count = to_OHE('story_id')
# job_position_count = to_OHE('job_position_cd')
# job_title_count = to_OHE('job_title')
jsons_name_type_count = to_OHE('jsons_name_type')




/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [13]:
values = set(train_and_val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values.reshape(-1))
producs_count = len(values)
d = dict(zip(list(values), range(1, len(values)+1)))
for f in ['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']:
    train[f]=train[f].map(lambda x:  d[x])
    val[f]=val[f].map(lambda x:  d[x])

/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
import keras.backend as K
def metric_loss(batch_size):
    d = np.sum(np.abs(valY))
    def the_loss(y_true, y_pred):
        return -(len(val.customer_id.values)/batch_size)*K.sum(y_true*y_pred)/d
    return(the_loss)


In [15]:
def reaction_to_class(reaction):
    if reaction=="dislike":
        return -10
    if reaction=="like":
        return 0.5
    if reaction=="view":
        return 0.1
    if reaction=="skip":
        return -0.1
trainY = np.array(list(train.event.map(lambda x: reaction_to_class(x)).values))
valY = np.array(list(val.event.map(lambda x: reaction_to_class(x)).values))

In [16]:
model_input_customer = Input((1,), name='customer_id')
customer = Embedding(customers_count, 4, input_length=1)(model_input_customer)
customer = Flatten()(customer)

model_input_story = Input((1,), name='story_id')
story = Embedding(stories_count, 2, input_length=1)(model_input_story)
story = Flatten()(story)

model_input_products = Input((7,), name='producs')
products = Embedding(producs_count, 1, input_length=7)(model_input_products)
products = Flatten()(products)



model_input_story_name = Input((1,), name='story_name_id')
story_name = Embedding(jsons_name_type_count, 2, input_length=1)(model_input_story_name)
story_name = Flatten()(story_name)

model_input_subset = Input((1,), name='model_input_subset')
subset = Embedding(20, 2, input_length=1)(model_input_subset)
subset = Flatten()(subset)


model_input_3 = Input((3,), name='customer_data')
model_input_4 = Input((2,), name='customer_data2')


x_other = Concatenate()([ products, story_name, model_input_3, model_input_4, subset])
x = Concatenate()([customer, story])


x = Dense(8, 
          activation='relu', 
         )(x)


x_other = Dense(4, 
          activation='relu', 
         )(x_other)




x = Concatenate()([x, x_other])



x = Dense(1, activation='tanh')(x)
model = Model([model_input_customer, 
               model_input_story, 
               model_input_story_name,
               model_input_subset,
               model_input_3,
               model_input_4, 
               model_input_products,
              ], x)





print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
customer_id (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
story_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
producs (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
story_name_id (InputLayer)      (None, 1)            0                                            
____________________________________________________________________________________________

In [17]:
# batch_size=320248
batch_size=64
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss=metric_loss(batch_size))


In [19]:


model.fit(
    shuffle=True,
        x = [train.customer_id.values,
             train.story_id.values,
             train.jsons_name_type.values, 
             train.subsets.values,
             
             train[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
             train[['reaction_hour', 'reaction_dayofweek']].values/30,
             train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
            ], 
          y = trainY,
          batch_size=batch_size,
          epochs=1,
          validation_data=([val.customer_id.values, 
                            val.story_id.values, 
                            val.jsons_name_type.values, 
                            val.subsets.values,
                            val[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
                            val[['reaction_hour', 'reaction_dayofweek']].values/30,
                            val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
                           ], valY),
         )

# 320248/320248 [==============================] - 13s 41us/step - loss: -0.6153 - val_loss: -0.3492
# 320248/320248 [==============================] - 13s 42us/step - loss: -0.7373 - val_loss: -0.3607
# 320248/320248 [==============================] - 14s 42us/step - loss: -0.7263 - val_loss: -0.3656 size 4 tanh
# 320248/320248 [==============================] - 14s 43us/step - loss: -0.7046 - val_loss: -0.3612 size 128 relu
# 320248/320248 [==============================] - 14s 44us/step - loss: -0.7110 - val_loss: -0.3455 size 128 lrelu
# 320248/320248 [==============================] - 14s 43us/step - loss: -0.6390 - val_loss: -0.3683 size 128 relu increase embs
# 320248/320248 [==============================] - 16s 48us/step - loss: -0.6319 - val_loss: -0.3705 job_position_cd


# 378578/378578 [==============================] - 61s 161us/step - loss: -0.4684 - val_loss: -0.4269
# 378578/378578 [==============================] - 15s 41us/step - loss: -0.5597 - val_loss: -0.4413
# 378578/378578 [==============================] - 2s 6us/step - loss: -0.5903 - val_loss: -0.4453

Train on 320248 samples, validate on 152893 samples
Epoch 1/1
320248/320248 [==============================] - 15s 46us/step - loss: -0.5910 - val_loss: -0.3494


In [114]:
# train_predict = model.predict([train.customer_id.values,
#              train.story_id.values,
#              0*train.jsons_name_type.values, 
#              train.job_title.values,
#              train.job_position_cd.values,
#              0*train.marital_status_cd.values,
#              train[['age', 'gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
#              train[[ 'customer_shows']].values/3000,
#              0*train[['transaction_amt_count', 'transaction_amt_min', 'transaction_amt_max']].values,
#              train[['reaction_hour', 'reaction_dayofweek']].values/30,
#              train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
#            ])
val_predict = model.predict([val.customer_id.values, 
                            val.story_id.values, 
                            val.jsons_name_type.values, 
                            val.subsets.values,
                            val[['gender_cd', 'children_cnt', 'first_session_dttm_year']].values,
                            val[['reaction_hour', 'reaction_dayofweek']].values/30,
                            val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values,
                            ])
# train_predict = model.predict(train.customer_id.values)
# val_predict = model.predict(val.customer_id.values)
-np.sum(valY*val_predict[:,0] )/np.sum(np.abs(valY))
# -0.3585501646600242

-0.35598356061260344

In [21]:
# np.sum(np.abs(trainY))

In [22]:
# -np.sum(trainY*train_predict[:,0])/np.sum(np.abs(trainY))

In [23]:
# -np.sum(trainY*train_predict[:,0] )/np.sum(np.abs(trainY))


In [24]:
-np.sum(valY*val_predict[:,0] )/np.sum(np.abs(valY))
# -0.22921870322866542
# -0.29499408463224436
# -0.3365937438413243
# -0.34325145608255186
# -0.35394938525789055


# -0.34627157632374744

-0.43540499300279123

In [25]:
val['predict'] = val_predict[:,0]

/home/ivan/PycharmProjects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
val.loc[~val.customer_id.isin(set(train.customer_id.values)),'predict'] =-1

/home/ivan/PycharmProjects/tks_stories/venv/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [27]:
-np.sum(valY*val['predict'])/np.sum(np.abs(valY))
# -0.26261507362945524
# -0.29364859095899104
# -0.3315410025698421

-0.4355258749860764